In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

In [2]:
def load_and_preprocess(file_path):
    data = pd.read_csv(file_path)

    X = data.drop("Revenue", axis=1)
    y = data["Revenue"]

    numerical_features = [
        "Marketing_Spend",
        "R&D_Spend",
        "Administration_Costs",
        "Number_of_Employees"
    ]

    categorical_features = ["Region"]

    preprocessor = ColumnTransformer(
        transformers=[
            ("num", StandardScaler(), numerical_features),
            ("cat", OneHotEncoder(handle_unknown="ignore"), categorical_features)
        ]
    )

    return X, y, preprocessor


def train_model(X_train, y_train, preprocessor):
    model = Pipeline(
        steps=[
            ("preprocessor", preprocessor),
            ("regressor", LinearRegression())
        ]
    )
    model.fit(X_train, y_train)
    return model


def evaluate_model(model, X_test, y_test):
    y_pred = model.predict(X_test)

    mae = mean_absolute_error(y_test, y_pred)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    r2 = r2_score(y_test, y_pred)

    print("\nModel Evaluation Metrics")
    print("------------------------")
    print(f"MAE  : {mae:.2f}")
    print(f"RMSE : {rmse:.2f}")
    print(f"R²   : {r2:.4f}")


def predict_revenue(model, input_data):
    input_df = pd.DataFrame([input_data])
    prediction = model.predict(input_df)
    print(f"Estimated Revenue: {prediction[0]:.2f}")


def main():
    file_path = "data.csv"

    X, y, preprocessor = load_and_preprocess(file_path)

    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42
    )

    model = train_model(X_train, y_train, preprocessor)

    evaluate_model(model, X_test, y_test)

    print("\nRevenue Prediction Scenarios")

    scenarios = [
        {
            "Marketing_Spend": 80000,
            "R&D_Spend": 90000,
            "Administration_Costs": 15000,
            "Number_of_Employees": 50,
            "Region": "North America"
        },
        {
            "Marketing_Spend": 45000,
            "R&D_Spend": 30000,
            "Administration_Costs": 20000,
            "Number_of_Employees": 35,
            "Region": "Europe"
        },
        {
            "Marketing_Spend": 120000,
            "R&D_Spend": 110000,
            "Administration_Costs": 25000,
            "Number_of_Employees": 80,
            "Region": "Asia"
        }
    ]

    for i, scenario in enumerate(scenarios, start=1):
        print(f"\nScenario {i}")
        predict_revenue(model, scenario)


if __name__ == "__main__":
    main()



Model Evaluation Metrics
------------------------
MAE  : 6648.40
RMSE : 8363.06
R²   : 0.9330

Revenue Prediction Scenarios

Scenario 1
Estimated Revenue: 77502.40

Scenario 2
Estimated Revenue: 39631.27

Scenario 3
Estimated Revenue: 106957.23
